In [1]:
from pyforest import * 
import warnings
warnings.filterwarnings("ignore")

# Read text dataset into pandas.

In [2]:
# read sms data into pandas..
data=pd.read_csv("spam.csv",encoding='latin-1')

<IPython.core.display.Javascript object>

In [3]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data=data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [5]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data=data.rename(columns = {'v1':'label', 'v2':'message'})

In [7]:
data.head(2)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [8]:
data.shape

(5572, 2)

In [9]:
data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [10]:
# converting target variable to numerical which is in categorical(ham/spam)
data["label_nu"]=data.label.map({'ham':'0', 'spam':'1'})

In [11]:
data.head(2)

,label,message,label_nu
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0


In [12]:
# Define X,y from data sets from data.
X=data.message
y=data.label_nu

In [13]:
print(X.shape)
print(y.shape)

(5572,)
(5572,)


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_test, X_train, y_test, y_train=train_test_split(X, y, random_state=7)

To convert given data into DTM.
1. Use X_train to creat Vocabulary
2. With created vocabulary, get DATM for X_train
3. With created vocabulary(using X_train data), get DTM for X_test

# Vectorize text dataset

In [16]:
# call the vectorizer
from sklearn.feature_extraction.text import CountVectorizer
toNumeric = CountVectorizer()

In [17]:
# creat vocabulary and creat DTM
toNumeric.fit(X_train)
X_train_dtm=toNumeric.transform(X_train)

In [18]:
X_train_dtm

<1393x3974 sparse matrix of type '<class 'numpy.int64'>'
	with 18453 stored elements in Compressed Sparse Row format>

In [19]:
# transform test data using fitted vocabulary into DTM
X_test_dtm=toNumeric.transform(X_test)
X_test_dtm

<4179x3974 sparse matrix of type '<class 'numpy.int64'>'
	with 48352 stored elements in Compressed Sparse Row format>

## Build and Evaluate the model

In [20]:
# Build and Evaluate the model
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [21]:
nb.fit(X_train_dtm, y_train)

MultinomialNB()

In [22]:
y_pred=nb.predict(X_test_dtm)

In [23]:
y_pred

array(['0', '0', '0', ..., '0', '1', '0'], dtype='<U1')

In [24]:
# Accuracy calculation
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.9784637473079684

In [25]:
nb.score(X_test_dtm, y_test)

0.9784637473079684

In [26]:
metrics.confusion_matrix(y_test, y_pred)

array([[3603,   11],
       [  79,  486]], dtype=int64)

In [27]:
#predict probability for test data instead of labels directly
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([5.23234459e-04, 1.25547426e-05, 3.31217178e-14, ...,
       1.35973775e-07, 9.99999999e-01, 1.52690003e-03])

In [28]:
# calculate AUC ROC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9678325685265267

In [29]:
## Multiple Model

In [30]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [31]:
lr.fit(X_train_dtm, y_train)

LogisticRegression()

In [32]:
y_pred_lr = lr.predict(X_test_dtm)
y_pred_lr

array(['0', '0', '0', ..., '0', '1', '0'], dtype=object)

In [33]:
lr.score(X_test_dtm, y_test)

0.9705671213208902

In [34]:
#predict probability for test data instead of labels directly
y_pred_prob_lr = lr.predict_proba(X_test_dtm)[:, 1]
y_pred_prob_lr

array([0.00995062, 0.00520373, 0.00326553, ..., 0.00233667, 0.97562171,
       0.02025752])

In [35]:
# Accuracy calculation
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_lr)

0.9705671213208902

In [36]:
# calculate AUC ROC
metrics.roc_auc_score(y_test, y_pred_prob_lr)

0.9828611447125485

# Fine Tune Vectorizer

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
toNumeric = TfidfVectorizer() # Can do with both TF-IDF and CV, no differance matters. Only accuracy can matter hardly.
toNumeric   # Default parameter for both.

TfidfVectorizer()

In [38]:
# Remove english stop words.
toNumeric=CountVectorizer(stop_words='english')
toNumeric.fit(X_train)
len(toNumeric.get_feature_names())

3742

In [39]:
toNumeric.get_feature_names()[-10:]

['åè10', 'åð', 'åôrents', 'ì_', 'ìï', 'ó_', 'û_', 'û_thanks', 'ûï', 'ûò']

In [40]:
# include 1 and 2-grams
toNumeric=TfidfVectorizer(ngram_range=(1,2))   # as same in countvectorizer.
toNumeric.fit(X_train)
len(toNumeric.get_feature_names())

17846

In [41]:
toNumeric.get_feature_names()[-10:]

['û_ ish',
 'û_thanks',
 'û_thanks and',
 'ûï',
 'ûò',
 'ûò am',
 'ûò even',
 'ûò hope',
 'ûò indeed',
 'ûò very']

In [42]:
toNumeric

TfidfVectorizer(ngram_range=(1, 2))

In [43]:
# Ignor terms that appear in more than 75% of the documents.
toNumeric=TfidfVectorizer(max_df=0.75)   # as same in countvectorizer.
toNumeric.fit(X_train)
len(toNumeric.get_feature_names())

3974

In [44]:
# Keep terms that appear in at least 5 Documents.
toNumeric=TfidfVectorizer(min_df=5)   # as same in countvectorizer.
toNumeric.fit(X_train)
len(toNumeric.get_feature_names())

645

# Word Cloud

In [1]:
# !pip install wordcloud      # can be needed to install in system.

In [46]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
data.head()

In [ ]:
wc=WordCloud()

In [ ]:
wc.generate(str(data["message"]))

In [ ]:
plt.figure(figsize=(20,10), facecolor='k')
plt.title("Most frequent word in SMS DataSet", fontsize=40, color='white')
plt.imshow(wc)
plt.grid(False)   # Hide grid lines
plt.show()

#  Lemmatization and stemming

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [ ]:
# Create an object of class PorterStemmer
porter=PorterStemmer()
lancaster=LancasterStemmer()

***Porter Stemmer***
- It uses set of rules to decide whether it is wise to strip a suffix.
- Quite often doesnot generate words which are in dictionary.
- It is known for it's simplicity and speed.


In [ ]:
# Provide a word to be stemmed
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))

**Sentence Stemming**

In [ ]:
sentence="Pthoners are very intelligent and work very pythonly and now they are pythoning their way to success"

In [ ]:
porter.stem(sentence)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def stemsentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

x=stemsentence(sentence)
print(x)

In [ ]:
# pip install textblob

In [ ]:
from textblob import TextBlob

In [ ]:
sent=TextBlob(sentence)

In [ ]:
print(' '.join([porter.stem(word) for word in sent.words]))

In [ ]:
# print(' '.join([porter.lemma(word) for word in sent.words]))

## Sentiment Calculation

In [ ]:
from textblob import TextBlob

In [ ]:
text="I hate any thing that goes in my ear"

In [ ]:
result = TextBlob(text)

In [ ]:
result.sentiment.polarity

In [ ]:
text="I am very happy"   #1.0

In [ ]:
result = TextBlob(text)

In [ ]:
result.sentiment.polarity

---------------------------------------------------- BE HAPPY ---------------------------------------------------------------